In [21]:
resp = '2023-11-10_07:16:34 — :jackattack5821!jackattack5821@jackattack5821.tmi.twitch.tv PRIVMSG #scump :Munchies hit'

values = resp.strip().split(' — ')
timestamp = values[0]
tw_message = values[1]
username = tw_message.split('!')[0].strip(':')

# user_handle = tw_message.split('!')[1].split(' ')[0]
channel = tw_message.split('!')[1].split(' ')[2]
chat_start = tw_message.find(channel)+len(channel)+2
message = tw_message[chat_start:]

In [22]:
import logging
import socket
import threading
import requests
import json
from google.cloud import bigquery as bq

# Create a logger instance
logger = logging.getLogger(__name__)

# # Initialize a BigQuery Client
client = bq.Client.from_service_account_json("key.json")

table_id = 'kirby42bot.twitch_chats.twitch_chat_log'

row_to_insert = {
        "timestamp": timestamp.replace('_', ' '),
        "channel": channel,
        "username": username,
        "message": message
    }

In [23]:
# Insert the row into the BigQuery table
table = client.get_table(table_id)
table

Table(TableReference(DatasetReference('kirby42bot', 'twitch_chats'), 'twitch_chat_log'))

In [24]:
client.insert_rows(table, [row_to_insert])

[]

In [27]:
import logging
import socket
import threading
import requests
import json
from google.cloud import bigquery as bq
from config import oauth_pw

# Create a logger instance
logger = logging.getLogger(__name__)

# twitch info
nickname = "Kirby42Bot"
token = oauth_pw
channels = ["#dashy", "#luxdigitalchurch", "#jatelive"]

# # Initialize a BigQuery Client
client = bq.Client.from_service_account_json("key.json")

table_id = 'kirby42bot.twitch_chats.twitch_chat_log'
table = client.get_table(table_id)


def connect_to_channel(nickname, token, channel):
    server = "irc.chat.twitch.tv"
    port = 6667
    sock = socket.socket()
    sock.connect((server, port))

    sock.send(f"PASS {token}\n".encode('utf-8'))
    sock.send(f"NICK {nickname}\n".encode('utf-8'))
    sock.send(f"JOIN {channel}\n".encode('utf-8'))

    # Create a custom logger
    logging.basicConfig(level=logging.DEBUG,
                        format='%(asctime)s — %(message)s',
                        datefmt='%Y-%m-%d_%H:%M:%S',
                        handlers=[logging.FileHandler('chat.log', encoding='utf-8')])

    while True:
        resp = sock.recv(2048).decode('utf-8')
        if resp.startswith('PING'):
            sock.send("PONG\n".encode('utf-8'))
        elif len(resp) > 0:
            logging.info(resp)
            # if resp.find('kirby42bot'):
            #     continue
            values = resp.strip().split(' — ')
            timestamp = values[0]
            tw_message = values[1]
            username = tw_message.split('!')[0].strip(':')
            # user_handle = tw_message.split('!')[1].split(' ')[0]
            channel = tw_message.split('!')[1].split(' ')[2]
            chat_start = tw_message.find(channel)+len(channel)+2
            message = tw_message[chat_start:]
            row_to_insert = {
                "timestamp": timestamp.replace('_', ' '),
                "channel": channel,
                "username": username,
                "message": message
            }

            logging.info(row_to_insert)
            # Insert the row into the BigQuery table
            errors = client.insert_rows(table, row_to_insert)

            if errors == []:
                logging.debug("New rows have been added.")
            else:
                logging.error(f"Error inserting row into BigQuery: {errors}")
                continue


for channel in channels:
    logger.debug(f'starting thread: {channel}')
    t = threading.Thread(target=connect_to_channel,
                         args=(nickname, token, channel))
    t.start()